In [116]:
import psycopg2
import os

# Setting up environment variables

In [117]:
os.environ['POSTGRE_USERNAME'] = 'postgres'
os.environ['POSTGRE_PASSWORD'] = 'admin'

# Load value from environment variables

In [118]:
USER = os.environ['POSTGRE_USERNAME']
PASSWORD = os.environ['POSTGRE_PASSWORD']

In [119]:
USER, PASSWORD

('postgres', 'admin')

----------

# Connecting to Database

In [120]:
conn = psycopg2.connect(database='db_Advanced_SQL_Application_Development',
                                       user=USER,
                                       password=PASSWORD,
                                       host='localhost',
                                       port=5432)

In [121]:
conn

<connection object at 0x0000003E7E1DD040; dsn: 'user=postgres password=xxx dbname=db_Advanced_SQL_Application_Development host=localhost port=5432', closed: 0>

-------------

# Parameterizing SELECT statements

In [122]:
# get all products type
def getProductsByType(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = 'SELECT * FROM products WHERE product_type = %s'
    cursor.execute(sql_string, [prod_type])
    return (cursor.fetchall())

In [123]:
# get oven type products
all_ovens = getProductsByType(conn, 'oven')

In [124]:
len(all_ovens)

20

In [125]:
all_ovens[:5]

[(1, 'Reflector oven', 'oven'),
 (2, 'Convection microwave', 'oven'),
 (15, 'Masonry oven', 'oven'),
 (24, 'Microwave oven', 'oven'),
 (32, 'Russian oven', 'oven')]

---------------

# Avoiding N+1 queries

### Let's say we want to get all product orders which include any type of ovens

In [126]:
# global cursor
cursor = conn.cursor()
fetchall_count = 0

for oven in all_ovens:
    oven_id = oven[0]
    sql_string = 'SELECT * FROM product_orders WHERE product_id = %s'
    
    cursor.execute(sql_string, [oven_id])
    oven_orders = cursor.fetchall()
    fetchall_count += 1
    
    print('fetchall_count: ', fetchall_count)
    print(oven_orders, "\n")

    
# destory global cursor
del cursor

fetchall_count:  1
[(4, 1, 1, Decimal('296.47')), (23, 1, 4, Decimal('30.99')), (57, 1, 4, Decimal('124.53')), (63, 1, 2, Decimal('131.79'))] 

fetchall_count:  2
[(5, 2, 2, Decimal('270.24')), (21, 2, 2, Decimal('196.39'))] 

fetchall_count:  3
[(9, 15, 3, Decimal('53.26')), (38, 15, 2, Decimal('299.62')), (41, 15, 2, Decimal('158.61'))] 

fetchall_count:  4
[(3, 24, 5, Decimal('131.20')), (32, 24, 5, Decimal('60.54')), (36, 24, 5, Decimal('93.13')), (49, 24, 4, Decimal('228.74')), (81, 24, 2, Decimal('283.90'))] 

fetchall_count:  5
[(24, 32, 3, Decimal('70.57'))] 

fetchall_count:  6
[(38, 33, 4, Decimal('236.11')), (40, 33, 1, Decimal('227.86')), (61, 33, 2, Decimal('45.44')), (82, 33, 3, Decimal('29.96'))] 

fetchall_count:  7
[(12, 34, 3, Decimal('251.16')), (28, 34, 4, Decimal('72.18')), (69, 34, 5, Decimal('73.06')), (71, 34, 2, Decimal('75.19'))] 

fetchall_count:  8
[(35, 37, 4, Decimal('211.71')), (49, 37, 1, Decimal('162.63')), (51, 37, 5, Decimal('271.15')), (61, 37, 2, De

**NOTE:**

In above query, we are like check against one item and get all orders of specific item. Drop them back at home.

Then again go back to store and check against another item and get all orders of that item. Drop them back at home again.

This make inefficient and we want to get all orders of all related items while querying and bring them back to home all together.

## more efficient way

In [127]:
def getProductOrdersByType(db_conn, product_type):
    cursor = db_conn.cursor()
    
    sql_string = 'SELECT po.* \
                        FROM product_orders po \
                        JOIN products p \
                        ON p.product_id = po.product_id  \
                        WHERE p.product_type = %s'
    
    cursor.execute(sql_string, [product_type])
    return (cursor.fetchall())

In [128]:
all_product_orders = getProductOrdersByType(conn, 'oven')

In [129]:
len(all_product_orders)

62

In [130]:
all_product_orders[:5]

[(2, 58, 5, Decimal('102.97')),
 (2, 80, 1, Decimal('181.54')),
 (3, 24, 5, Decimal('131.20')),
 (4, 1, 1, Decimal('296.47')),
 (5, 2, 2, Decimal('270.24'))]

----------

# Challenge
- write a function that executes an SQL query to reterive all product orders.
- type: fryer
- quanity greater than 3

In [152]:
def getAllProductOrdersByType(db_conn, product_type, quantity_greater_than):
    
    cursor = db_conn.cursor()
    sql_string = 'SELECT po.* \
                        FROM product_orders po \
                        JOIN products p \
                        ON p.product_id = po.product_id \
                        WHERE p.product_type = %s \
                        AND po.quantity > %s'
    
#     sql_string = 'SELECT po.* \
#                         FROM product_orders po \
#                         JOIN products p \
#                         ON p.product_id = po.product_id  \
#                         WHERE p.product_type = %s \
#                         AND po.quantity > 3'
    
    cursor.execute(sql_string, [product_type, quantity_greater_than])
    return (cursor.fetchall())

In [153]:
all_product_orders = getAllProductOrdersByType(conn, 'fryer', 3)

In [154]:
len(all_product_orders)

7

In [155]:
all_product_orders

[(1, 3, 5, Decimal('72.69')),
 (11, 3, 4, Decimal('296.83')),
 (18, 26, 5, Decimal('253.36')),
 (53, 3, 5, Decimal('161.05')),
 (53, 17, 4, Decimal('125.62')),
 (59, 54, 5, Decimal('272.47')),
 (70, 3, 4, Decimal('261.91'))]